In [96]:
pwd

u'/'

In [97]:
ls

Applications/              etc@
Library/                   home/
Network/                   installer.failurerequests
PostInstall.sh*            net/
System/                    opt/
Users/                     private/
Volumes/                   sbin/
bin/                       tmp@
cores/                     usr/
dev/                       var@


In [98]:
import pyspark


In [99]:
sc

In [100]:
rdd = sc.textFile('/Users/arnavsomani/spark-2.1.0-bin-hadoop2.7/README.md')
testRdd = sc.parallelize([1,2,3,4,5])
rdd.take(10)

[u'# Apache Spark',
 u'',
 u'Spark is a fast and general cluster computing system for Big Data. It provides',
 u'high-level APIs in Scala, Java, Python, and R, and an optimized engine that',
 u'supports general computation graphs for data analysis. It also supports a',
 u'rich set of higher-level tools including Spark SQL for SQL and DataFrames,',
 u'MLlib for machine learning, GraphX for graph processing,',
 u'and Spark Streaming for stream processing.',
 u'',
 u'<http://spark.apache.org/>']

In [101]:
testRdd.collect()

[1, 2, 3, 4, 5]

In [102]:
wc = rdd.flatMap(lambda line: line.split()) \
           .map(lambda x:(x.lower(),1)) \
           .groupByKey() \
           .mapValues(lambda values: sum(values))


In [103]:
wc.take(2)

[(u'when', 1), (u'alternatively,', 1)]

In [104]:
wc = rdd.flatMap(lambda line: line.split()) \
           .map(lambda x:(x.lower(),1)) \
           .reduceByKey(lambda x,y: x+y)
    

wc.take(2)

[(u'when', 1), (u'alternatively,', 1)]

In [105]:
wc.top(3,key=lambda x:x[1])

[(u'the', 25), (u'to', 19), (u'spark', 16)]

In [106]:
cd ..

/


In [107]:
pwd

u'/'

In [108]:
SAT_FN = 'SAT_Results.csv'
HSD_FN = 'DOE_High_School_Directory_2014-2015.csv'

In [109]:
df1 = spark.read \
           .format('com.databricks.spark.csv')\
           .option('header','true') \
           .option('inferSchema','true') \
           .option('parserLib','UNIVOCITY') \
           .load(HSD_FN) 

In [110]:
sat = sc.textFile(SAT_FN, use_unicode=False).cache()
#b = sat.collect()
#b
sat.take(2)

['DBN,SCHOOL NAME,Num of SAT Test Takers,SAT Critical Reading Avg. Score,SAT Math Avg. Score,SAT Writing Avg. Score',
 '02M047,47 THE AMERICAN SIGN LANGUAGE AND ENGLISH SECONDARY SCHOOL,16,395,400,387']

In [111]:
sat.take(2)

['DBN,SCHOOL NAME,Num of SAT Test Takers,SAT Critical Reading Avg. Score,SAT Math Avg. Score,SAT Writing Avg. Score',
 '02M047,47 THE AMERICAN SIGN LANGUAGE AND ENGLISH SECONDARY SCHOOL,16,395,400,387']

In [112]:
list(enumerate(sat.first().split(',')))

[(0, 'DBN'),
 (1, 'SCHOOL NAME'),
 (2, 'Num of SAT Test Takers'),
 (3, 'SAT Critical Reading Avg. Score'),
 (4, 'SAT Math Avg. Score'),
 (5, 'SAT Writing Avg. Score')]

In [113]:
#sat = sat.repartition(10)

In [114]:
#sat.getNumPartitions()

In [115]:
#sat.filter(lambda x: not x.startwith('DBN,SCHOOL')).take(4)

In [116]:
def extractScores(partId,list_of_records):
    if partId==0:
        list_of_records.next()
    import csv
    reader = csv.reader(list_of_records)
    for row in reader:
        if row[2]!='s':
            (dbn,takers,score) = (row[0],int(row[2]),int(row[4]))
            yield dbn,(score*takers,takers)

In [117]:
satScores = sat.mapPartitionsWithIndex(extractScores)

In [118]:
#a = satScores.collect()


In [119]:
schools = sc.textFile(HSD_FN, use_unicode=False).cache()

In [120]:
#list(enumerate(school.first().split(',')))

In [121]:
def extractSchools(partId,list_of_records):
    if partId==0:
        list_of_records.next()
    import csv
    reader = csv.reader(list_of_records)
    for row in reader:
        if len(row)==58 and row[17].isdigit():
            (dbn,boro,total_students) = (row[0],row[2],int(row[17]))
            if total_students>500:
                yield(dbn,boro)
            

In [122]:
largeSchools = schools.mapPartitionsWithIndex(extractSchools)

In [123]:
largeSchools.take(10)

[('01M450', 'Manhattan'),
 ('01M539', 'Manhattan'),
 ('01M696', 'Manhattan'),
 ('02M374', 'Manhattan'),
 ('02M400', 'Manhattan'),
 ('02M408', 'Manhattan'),
 ('02M412', 'Manhattan'),
 ('02M413', 'Manhattan'),
 ('02M416', 'Manhattan'),
 ('02M418', 'Manhattan')]

In [124]:
scores = largeSchools.join(satScores).values()

In [125]:
scores.take(3)

[('Bronx', (23069, 59)),
 ('Staten Island', (52216, 107)),
 ('Bronx', (16317, 49))]

In [126]:
scores.reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1])) \
    .mapValues(lambda x: x[0]/x[1]) \
    .collect()

[('Bronx', 470),
 ('Manhattan', 514),
 ('Brooklyn', 487),
 ('Staten Island', 477),
 ('Queens', 474)]